### Vetorizar documento

In [ ]:
import re

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
df = pd.read_csv('../data/data.csv')

In [ ]:
# Download necessário para a NLTK
nltk.download('stopwords')  # Baixa a lista de stop words (palavras comuns) para uso no processamento de texto
nltk.download('punkt')  # Baixa o tokenizer Punkt, necessário para a tokenização de frases

# Carregar o modelo de português para o spaCy
!python -m spacy download pt_core_news_sm

In [ ]:
def remove_noise(text):

    text = re.sub(r'http\S+', '', text)

    # Converte para minúsculas
    text = text.lower()

    # Remove pontuação
    # []: colchetes são usados para definir uma classe de caracteres.
    # ^: quando usado no início de uma classe de caracteres, o ^ nega a classe, ou seja, seleciona tudo que não está na classe.
    # \w: corresponde a qualquer caractere alfanumérico (letras e números, incluindo o caractere de sublinhado _)
    # \s: corresponde a qualquer espaço em branco (espaços, tabulações, quebras de linha).
    text = re.sub(r'[^\w\s]', '', text)

    # Remove underlines
    text = re.sub(r'_+', '', text)

    # Remove números
    # \d: corresponde a qualquer dígito (de 0 a 9).
    # +: significa “um ou mais” do elemento precedente. Portanto, \d+ corresponde a uma sequência de um ou mais dígitos consecutivos.
    text = re.sub(r'\d+', '', text)

    return text

In [ ]:
def remove_stopwords(text):
    # Obtém a lista de stopwords em português usando o NLTK e as converte para um conjunto para melhorar a eficiência da busca
    stop_words = set(stopwords.words('portuguese'))

    # Divide o texto em palavras, remove as stopwords e então junta as palavras restantes de volta em uma string
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

In [ ]:
data = df[df.resultado != 'NÃO DEFINIDO']
data

In [ ]:
data['detalhamento'] = data['detalhamento'].apply(remove_noise)
data.head()

In [ ]:
data.iloc[1]['detalhamento']

In [ ]:
data['detalhamento'] = data['detalhamento'].apply(remove_stopwords)
data.head()

In [ ]:
text = data['detalhamento'][210]
text

In [ ]:
import spacy
nlp = spacy.load('pt_core_news_sm')

In [ ]:
def tokenize_text(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    return tokens


data['tokens'] = data['detalhamento'].apply(tokenize_text)

In [ ]:
data['tokens'][210]

## Stemming e Lemmatização

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# def apply_stemming(tokens):
#     stemmer = PorterStemmer()
#     stemmed_tokens = [stemmer.stem(token) for token in tokens]
#     return stemmed_tokens

In [ ]:
# data['stemming'] = data['tokens'].apply(apply_stemming)
# data['stemming'][267]

In [ ]:
# stemmer = PorterStemmer()

# # Tokeniza a frase em palavras
# palavras = word_tokenize(data['detalhamento'][267])

# # Aplica o stemming a cada palavra
# stemmed_words = [stemmer.stem(word) for word in palavras]

# print("Frase original:", data['detalhamento'][267])
# print("Palavras após Stemming:", stemmed_words)
# print()

In [ ]:
def apply_lemming(doc):
    doc = ' '.join(doc)

    nlp = spacy.load("pt_core_news_sm")

    doc = nlp(doc)

    lemmatized_words = [token.lemma_ for token in doc]

    return lemmatized_words

In [ ]:
data['clean_version'] = data['tokens'].apply(apply_lemming)

In [ ]:
%pip install scikit-learn

In [ ]:
x = data['clean_version']
y = data['resultado']

print(len(x),  len(y))

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

x_train_flattened = [item for sublist in x_train for item in sublist]

vect = CountVectorizer()
vect.fit(x_train_flattened)

In [ ]:
x_train_dtm = vect.transform(x_train_flattened)

x_test_flattened = [item for sublist in x_test for item in sublist]

x_test_dtm = vect.transform(x_test_flattened)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_transformer = TfidfTransformer()


tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.fit_transform(x_train_dtm)

x_train_tfidf_dense = x_train_tfidf.toarray()

print(x_train_tfidf_dense)


vectorizer_tfidf = TfidfVectorizer()
tfidf_matrix = vectorizer_tfidf.fit_transform(x_train_flattened)

print(tfidf_matrix.toarray())
print(vectorizer_tfidf.get_feature_names_out())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer()
# Ajustar e transformar os documentos em uma matriz TF-IDF
X_tfidf = vectorizer_tfidf.fit_transform(
    data['clean_version'].apply(lambda x: ' '.join(x)))
vocab = vectorizer_tfidf.get_feature_names_out()
print("Representação TF-IDF:\n", X_tfidf.toarray())
print("Vocabulário TF-IDF:\n", vocab)

print("Vocabulário TF-IDF:", vocab)
# Imprime a matriz TF-IDF com rótulos de linha e coluna
print("Matriz TF-IDF:")
print("Documento ", end="")
for i, doc in enumerate(X_tfidf.toarray()):
    print(f"Documento {i+1}:", end="   ")
    for word, tfidf in zip(vocab, X_tfidf[i].toarray()[0]):
        if tfidf == 0:
            continue
        print(f"{word}: {tfidf:.4f}")
    print()

In [ ]:
data.count()

## Vetorizar dataframe using TFiDF

In [ ]:
data.fillna('', inplace=True)

In [ ]:
data['detalhamento'] = data['clean_version'].apply(lambda x: ' '.join(x))

## Nuvem de Palavras

In [ ]:
pip install wordcloud

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import nltk

# Baixar as stop words se necessário
nltk.download('stopwords')
stop_words = stopwords.words('portuguese')

# Preencher valores nulos e preparar a coluna 'detalhamento' para o TF-IDF
data.fillna('', inplace=True)
data['detalhamento'] = data['clean_version'].apply(lambda x: ' '.join(x))

# Criar o vetor TF-IDF usando stop words em português
vectorizer_tfidf = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer_tfidf.fit_transform(data['detalhamento'])

# Obter o vocabulário e as somas dos valores TF-IDF para cada palavra
vocab = vectorizer_tfidf.get_feature_names_out()
tfidf_sum = tfidf_matrix.sum(axis=0).tolist()[0]  # Soma os TF-IDF ao longo de todos os documentos

# Mapear cada palavra para seu valor total de TF-IDF
word_scores = dict(zip(vocab, tfidf_sum))

# Gerar a nuvem de palavras com os pesos de TF-IDF
wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_scores)

# Exibir a nuvem de palavras
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()



In [ ]:
data_tfidf = data.drop(columns=['requerente_8', 'lemming', 'stemming', 'tokens', 'pdf_file_path', 'resultado'])
columns = data_tfidf.columns

for column in columns:
    print(f'== {column} ==')
    if data_tfidf[column].str.strip().any():
        tfidf_matrix = vectorizer_tfidf.fit_transform(data_tfidf[column])

        data_tfidf[column] = tfidf_matrix.toarray().mean(axis=1)
    else:
        data_tfidf[column] = df[column].replace('', 0)

data_tfidf

In [ ]:
data_tfidf[100:110]

In [ ]:
data.to_csv('../data/data_tfidf.csv', index=False)